In [18]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '발화 스타일 예측', '끊어읽기']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['id']
            text = line['recite_src']['text']
            styles = ['/'.join(style.values()) for style in line['recite_src']['styles']]
            sentences = [sentence['origin_text'] for sentence in line['sentences']]
            sentences_style = ['/'.join(s.values()) for sentence in line['sentences'] for s in sentence['styles']]
            sentences_ptr = [sentence['voice_piece']['ptr'] for sentence in line['sentences']]
            
            literature = line['recite_src']['literature']
            reciter = line['reciter']
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': text}
            ## preprocess data from line
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### 발화 스타일 예측
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['reciter'] = reciter
            data['input']['literature'] = literature
            data['input']['sentences'] = sentences
            data['output']['styles'] = sentences_style
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 발화 스타일 예측 end


            #### 끊어읽기
            ## preprocess data from line
            data['input']['sentences'] = [{'sentence': sentence, 'style': style} for sentence, style in zip(sentences, data['output'].pop('styles'))]
            data['output']['ptr'] = sentences_ptr
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 끊어읽기 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  2.51it/s]
